In [1]:
pip install opencv-python==3.4.18.65

  Using cached opencv_python-3.4.18.65-cp36-abi3-macosx_10_15_x86_64.whl.metadata (18 kB)
Using cached opencv_python-3.4.18.65-cp36-abi3-macosx_10_15_x86_64.whl (44.3 MB)
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.9.0
ERROR: Cannot uninstall opencv-python 4.9.0, RECORD file not found. Hint: The package was installed by conda.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import pyautogui
import matplotlib.pyplot as plt
import apriltag
import argparse

In [ ]:
# init part
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

# Setup SimpleBlobDetector parameters.
params = cv2.SimpleBlobDetector_Params()

#***DEFAULT PARAMETERS***#

# Change thresholds (0 to 255) 
params.minThreshold = 0
params.maxThreshold = 255

#Area - area in terms of pixels
params.filterByArea = True
params.minArea = 300
params.maxArea = 10000
      
#Circularity - measures how close the blob to a circle
params.filterByCircularity = True
params.minCircularity = 0.5 #"cicularity" of a square = 0.785 , circle = 1.000
params.maxCircularity = 1.00 

#Color - 0 (select darker) to  255 (select lighter)
params.filterByColor = False
#params.minColor = 0 
#params.maxColor = 255 

#Convexity - (Area of the Blob / Area of it’s convex hull)
params.filterByConvexity = False
params.minConvexity = 0.87
params.maxConvexity = 1.00

#Inertia - how elongated a shape is, line = 0.000, circle = 1.000,
params.filterByInertia = False
params.minInertiaRatio = 0.95
params.maxInertiaRatio = 1.00

#Repeatability
params.minRepeatability=1

font = cv2.FONT_HERSHEY_SIMPLEX


# def blob_process(img, threshold, detector):
#     gray_frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     _, img = cv2.threshold(gray_frame, threshold, 255, cv2.THRESH_BINARY)
#     img = cv2.erode(img, None, iterations=2)
#     img = cv2.dilate(img, None, iterations=4)
#     img = cv2.medianBlur(img, 5)
#     keypoints = detector.detect(img)
#     print(keypoints)
#     return keypoints


def nothing(x):
    pass

'''
def update_values(x):
    global font, cap, bgr_track
    frame = cv2.putText(frame, f"{bgr_track['R']}", (30, 330), font, 0.5, (0,0,0), 1)    # Fill previous text pixels with black
    frame = cv2.putText(frame, f"{x}", (30, 330), font, 0.5, (255,255,255), 1)   # Put new text with color white
    bgr_track['R'] = x
'''



def main():
    screen_w, screen_h = pyautogui.size()
    cap = cv2.VideoCapture(0)
    #Video feed window
    #lfw = 'Live-Feed'
    #cv2.namedWindow(lfw)
    
    #parameter control window
    pcw = 'Parameter controls'
    cv2.namedWindow(pcw)
    
    
    threshold = 20
    areaParam = 1000
    circularityParam = 100
    colorParam = 20
    convexityParam = 100
    inertiaParam = 100

    '''
    # create switch for ON/OFF functionality

    cv2.createTrackbar('Threshold', pcw, threshold, 100, nothing)
    cv2.createTrackbar('Area', pcw, areaParam, 2000, nothing)
    cv2.createTrackbar('Circularity', pcw, circularityParam, 100, nothing)
    cv2.createTrackbar('Color', pcw, colorParam, 255, nothing)
    cv2.createTrackbar('Convexity', pcw, convexityParam, 100, nothing)
    cv2.createTrackbar('Inertia', pcw, inertiaParam, 100, nothing)
    
    # create switch for ON/OFF functionality
    
    areaSwitch = 'Area OFF/ON'
    cv2.createTrackbar(areaSwitch, pcw, 0, 1, nothing)
    circularitySwitch = 'Color OFF/ON'
    cv2.createTrackbar(circularitySwitch, pcw, 0, 1, nothing)
    colorSwitch = 'Color OFF/ON'
    cv2.createTrackbar(colorSwitch, pcw, 0, 1, nothing)
    convexitySwitch = 'Convexity OFF/ON'
    cv2.createTrackbar(convexitySwitch, pcw, 0, 1, nothing)
    inertiaSwitch = 'Inertia OFF/ON'
    cv2.createTrackbar(inertiaSwitch, pcw, 0, 1, nothing)
    '''
    
    while True:
        _, frame = cap.read()

        #convert to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        print("[INFO] detecting AprilTags...")
        options = apriltag.DetectorOptions(families="tag36h11")
        detector = apriltag.Detector(options)
        results = detector.detect(gray_frame)
        print("[INFO] {} total AprilTags detected".format(len(results)))

        # loop over the AprilTag detection results
        for r in results:
            # extract the bounding box (x, y)-coordinates for the AprilTag
            # and convert each of the (x, y)-coordinate pairs to integers
            (ptA, ptB, ptC, ptD) = r.corners
            ptB = (int(ptB[0]), int(ptB[1]))
            ptC = (int(ptC[0]), int(ptC[1]))
            ptD = (int(ptD[0]), int(ptD[1]))
            ptA = (int(ptA[0]), int(ptA[1]))

            # draw the bounding box of the AprilTag detection
            cv2.line(frame, ptA, ptB, (0, 255, 0), 2)
            cv2.line(frame, ptB, ptC, (0, 255, 0), 2)
            cv2.line(frame, ptC, ptD, (0, 255, 0), 2)
            cv2.line(frame, ptD, ptA, (0, 255, 0), 2)

            # draw the center (x, y)-coordinates of the AprilTag
            (cX, cY) = (int(r.center[0]), int(r.center[1]))
            cv2.circle(frame, (cX, cY), 5, (0, 0, 255), -1)

            # draw the tag family on the image
            tagFamily = r.tag_family.decode("utf-8")
            cv2.putText(frame, tagFamily, (ptA[0], ptA[1] - 15),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            print("[INFO] tag family: {}".format(tagFamily))



        '''
        # *** Get parameters from trackbars ***
        threshold = r = cv2.getTrackbarPos('Threshold', pcw)
        areaParam = cv2.getTrackbarPos('Area', pcw)
        circularityParam = cv2.getTrackbarPos('Circularity', pcw)
        colorParam = cv2.getTrackbarPos('Color', pcw)
        convexityParam = cv2.getTrackbarPos('Convexity', pcw)
        inertiaParam = cv2.getTrackbarPos('Inertia', pcw)
        '''
        
        #Add text to frame to update displayed parameters
        frame = cv2.rectangle(frame,(0,0),(2000,60),(0,0,0),-5) #black background for white text
        frame = cv2.putText(frame, f'Min Area: {params.minArea}', (10, 15), font, 0.5, (255,255,255), 1)
        frame = cv2.putText(frame, f'Max Area: {params.maxArea}', (10, 55), font, 0.5, (255,255,255), 1)

        frame = cv2.putText(frame, f'Min Circ: {params.minCircularity}', (200, 15), font, 0.5, (255,255,255), 1)
        frame = cv2.putText(frame, f'Max Circ: {params.maxCircularity}', (200, 55), font, 0.5, (255,255,255), 1)

        frame = cv2.putText(frame, f'Color: {colorParam}', (400, 15), font, 0.5, (255,255,255), 1)
        frame = cv2.putText(frame, f'Convexity: {convexityParam}', (600, 15), font, 0.5, (255,255,255), 1)
        frame = cv2.putText(frame, f'Inertia: {inertiaParam}', (800, 15), font, 0.5, (255,255,255), 1)
        frame = cv2.putText(frame, f'Threshold: {inertiaParam}', (1000, 15), font, 0.5, (255,255,255), 1)

        
        if (threshold < 20):
            threshold = 20
        #print("Threshold Raw")
        #print(threshold)
        threshold = threshold/100
        #print("Threshold Percent")
        #print(threshold)

        #Update Parameters
        params.minArea = 1000
        params.maxArea = 5000
        params.minCircularity = 0.5 #circularityParam/100
        params.maxCircularity = 1 #circularityParam/100
        #params.minColor = 0 # no minColor
        #params.maxColor = 255 # no maxColor
        params.minConvexity = 0
        params.maxConvexity = 1
        params.minInertiaRatio = 0 #inertiaParam/100 
        params.maxInertiaRatio = 1 #inertiaParam/100
        params.minThreshold = 0
        params.maxThreshold = 255

        

        # Create a detector with the parameters

        ver = (cv2.__version__).split('.')
        if int(ver[0]) < 3 :
          detector = cv2.SimpleBlobDetector(params)
        else : 
          detector = cv2.SimpleBlobDetector_create(params)

        keypoints = detector.detect(frame)
        '''
        print("Keypoints")
        print(type(keypoints))
        print(keypoints)
        print(cv2.KeyPoint_convert(keypoints))
 
        print(type(points))
        print(type(pointsTuple))
        '''

        points = cv2.KeyPoint_convert(keypoints)
        pointsTuple = tuple(map(tuple,points))

        #print(keypointsfirst)
        #print(keypoints[1])
        


                     
        if pointsTuple:
            xcoord = int(pointsTuple[0][0])
            ycoord = int(pointsTuple[0][1])
            #print(xcoord)
            #print(ycoord)
            #screen_x = screen_w * xcoord
            #screen_y = screen_h * ycoord
            #print(screen_x)
            #print(screen_y)
            #TODO turn on
            #pyautogui.moveTo(xcoord,ycoord)

            #frame = cv2.putText(frame, 'C', (xcoord, ycoord), font, 0.5, (0,0,255), 1)
            frame = cv2.circle(frame,(xcoord, ycoord), 5, (0,0,255), 5)



        eye = cv2.drawKeypoints(frame, keypoints, frame, (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        #print("Eye")
        #print(type(eye))
        #print(eye)

        #print("Min area")
        #print(params.minArea)

        cv2.imshow('image', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()

[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 t

/private/var/folders/8v/p7szzjzd7bx0jz_vlzbg0m700000gn/T/pip-install-wbqxel8q/apriltag_0c40b5eef5104d16ae78977fd022c006/core/contrib/apriltag_quad_contour.c:467: warning: count < 8 :(


[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO]

/private/var/folders/8v/p7szzjzd7bx0jz_vlzbg0m700000gn/T/pip-install-wbqxel8q/apriltag_0c40b5eef5104d16ae78977fd022c006/core/contrib/apriltag_quad_contour.c:467: warning: count < 8 :(


[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 total AprilTags detected
[INFO] detecting AprilTags...
[INFO] 0 t